# Assignment 2

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning sessions. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import specific objects
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from ISLP import confusion_table
from ISLP import load_data

### Question 1: Classification using KNN

We'll now use the `Caravan` dataset from the `ISLP` package. (You may use `Caravan.describe()` to review details of the dataset.) In this dataset, the response variable of interest is `Purchase`, which indicates if a given customer purchased a caravan insurance policy. We will simultaneously use all other variables in the dataset to predict the response variable.

In [2]:
# Load the "Caravan" dataset using the "load_data" function from the ISLP package
Caravan = load_data('Caravan')
#val = Caravan['Purchase'].describe()
#val['unique']
print(Caravan)
# desc = Caravan.describe()
# # print(desc.columns)
# # Add your code here
# Caravan.columns.difference(desc.columns)


      MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  MGODOV  \
0          33         1        3         2         8       0       5       1   
1          37         1        2         2         8       1       4       1   
2          37         1        2         2         8       0       4       2   
3           9         1        3         3         3       2       3       2   
4          40         1        4         2        10       1       4       1   
...       ...       ...      ...       ...       ...     ...     ...     ...   
5817       36         1        1         2         8       0       6       1   
5818       35         1        4         4         8       1       4       1   
5819       33         1        3         4         8       0       6       0   
5820       34         1        3         2         8       0       7       0   
5821       33         1        3         3         8       0       6       1   

      MGODGE  MRELGE  ...  APERSONG  AG

Before fitting any model, it is essential to understand our data. Answer the following questions about the `Caravan` dataset (Hint: use `print` and `describe`):  
_(i)_ How many observations (rows) does the dataset contain?    
_(ii)_ How many variables (columns) does the dataset contain?    
_(iii)_ What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable?    
_(iv)_ How many predictor variables do we have (Hint: all variables other than `Purchase`)?  

In [3]:
# (i) How many observations (rows) does the dataset contain? (todo: is this right though?)
print(f"observations (rows) in caravan data model: {Caravan.shape[0]}")
# (ii) How many variables (columns) does the dataset contain?  
print(f"variables (columns) in caravan data model: {Caravan.shape[1]}")
# (iii) What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable?
print(f"variable type: Factor (categorical), levels: {Caravan['Purchase'].describe()['unique']}")
# (iv) How many predictor variables do we have (Hint: all variables other than `Purchase`)?  
print(f"predictor variables (columns not named 'Purchase') in caravan data model: {len(Caravan.describe().columns)}")

observations (rows) in caravan data model: 5822
variables (columns) in caravan data model: 86
variable type: Factor (categorical), levels: 2
predictor variables (columns not named 'Purchase') in caravan data model: 85


Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the `scaler` method, provided as follows:

In [67]:
# Select predictors (excluding the 86th column)
predictors = Caravan.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head(1)) # make this one to not take up as much room on my screen.

    MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK    MGODPR  \
0  0.680906  -0.27258  0.406697 -1.216964  0.779405 -0.694311  0.217444   

     MGODOV    MGODGE   MRELGE  ...   ALEVEN  APERSONG   AGEZONG  AWAOREG  \
0 -0.068711 -0.161816  0.42767  ... -0.20291 -0.073165 -0.081055 -0.05992   

     ABRAND   AZEILPL  APLEZIER   AFIETS   AINBOED  ABYSTAND  
0  0.764971 -0.022706  -0.07365 -0.15062 -0.087348 -0.118816  

[1 rows x 85 columns]


_(v)_ Why is it important to standardize the predictor variables?  
_(vi)_ Why did we elect not to standard our response variable `Purchase`?  


### (v) Why is it important to standarize the predictor variables?
Certain variables are significantly larger than others. Variables on a large scale will have a much larger affect on the distance between the observations. These larger values can have an outsized influence on the KNN classification results. Standardizing them via scaling allows all variables to be on a comparable scale. So a a larger variable will still have an effect, but not an outsized one.

### (vi) Why did we elect not to standard our response variable `Purchase`?
That's the value we're trying to predict. We shouldn't be using Purchase at all in our data, hence we drop it.


_(vii)_ A second essential step is to set a random seed. Do so below (Hint: use the `random.seed` function). Why is setting a seed important? Is the particular seed value important? Why or why not?

In [64]:
# Add your code here
# This affects the np.random method below. Using `train_test_split` requires the seed to be per method.
np.random.seed(42)
print("""
While KNN's algorithm is running, there is the potential for multiple neighbors to be tied during a measurement. If this occurs, Scikit will randomly break the tie so it can continue. By setting
the seed, we force scikit to use a consistent random number, making our results reproducible.
      
The seed value isn't particularly important. It only needs to stay constant across all runs for a good comparison.
""")


While KNN's algorithm is running, there is the potential for multiple neighbors to be tied during a measurement. If this occurs, Scikit will randomly break the tie so it can continue. By setting
the seed, we force scikit to use a consistent random number, making our results reproducible.
      
The seed value isn't particularly important. It only needs to stay constant across all runs for a good comparison.



_(viii)_ A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. Extend the code to create a non-overlapping test set for the predictors and response variables.

In [121]:
# Create a random vector of True and False values
split = np.random.choice(
    [True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25]
)
# Define the training set for X (predictors)
training_X = predictors_standardized[split]

# Define the training set for Y (response)
training_Y = Caravan.loc[split, "Purchase"]

# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the testing set for Y (response)
testing_Y = Caravan.loc[~split, "Purchase"]

#Extended to include the random state seed, the NP.  Avoids needing to check for outright overlapping values
X_train, X_test, Y_train, Y_test = train_test_split(
    predictors_standardized, Caravan.Purchase, test_size=0.25, random_state=42
)

[ True False  True ...  True  True False]


_(ix)_ We are finally set to fit the KNN model. In Python, we can use the `KNeighborsClassifier()` function. Fit the KNN with k=1. (You may review arguments to knn by typing `help(knn.fit)`). 

In [113]:
# Add your code here
knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(training_X, training_Y)
# knn.score(testing_X, testing_Y)
knn.fit(X_train, Y_train)
knn.score(X_test, Y_test)
# confusion_table(knn_pred, testing_Y)

0.8846153846153846

Using your fit model, answer the following questions:   
_(x)_ What is the prediction accuracy? (Hint: use the `score` method, and compare your model to `testing_Y`)  
_(xi)_ What is the predictor error ? (Hint: compute it from the accuracy)

In [115]:
# prediction accuracy rate
print(f"prediction accuracy rate: {knn.score(X_test, Y_test)*100} %")

prediction accuracy rate: 88.46153846153845 %


In [117]:
# prediction error rate
#print(f"prediction error rate: {(1 - knn.score(X_test, Y_test)) * 100} %")
print(f"prediction error rate: {np.mean(Y_test != knn.predict(X_test)) * 100} %")
# confusion_table(knn_pred, testing_Y)

prediction error rate: 11.538461538461538 %


_(xii)_ How does this prediction error/accuracy compare to what could be achieved via random guesses? To answer this, consider the percent of customers in the `Caravan` dataset who actually purchase insurance, computed below:

In [120]:
# Calculate the percentage of customers who purchase insurance
percentage_purchase = (
    Caravan["Purchase"].eq("Yes").sum() / Caravan["Purchase"].eq("No").sum()
) * 100

print(percentage_purchase)
print(
    """
      A random prediction would give, on average, a 50% accuracy/error rate. The model above is significantly more accurate.
    """
)

6.357325538911216

      A random prediction would give, on average, a 50% accuracy/error rate. The model above is significantly more accurate.
    


_(xiii)_ Fit a second KNN model, with $K=3$. Does this model perform better (i.e., have higher accuracy, compared to a random guess)?

In [119]:
# Add your code here
knn2 = KNeighborsClassifier(n_neighbors=3)
knn2.fit(X_test, Y_test)
knn_pred2 = knn.predict(X_test)
knn2_score = knn2.score(X_test, Y_test)

print(f"model with a K (nearest neighbors) of 3 has an accuracy score of {knn2_score * 100} %. This is higher than our first model by nearly 2%, and is significantly better than a random guess.")

model with a K (nearest neighbors) of 3 has an accuracy score of 93.95604395604396 %. This is higher than our first model by nearly 2%, and is significantly better than a random guess.


# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Classification using KNN|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applied_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
